### CUT AND SAVE IMAGES

In [1]:
import os
import torch
import torch.nn as nn
import random
import math
import numpy as np
import torchvision 
from torchvision import transforms, datasets
import torchvision.datasets as dataset
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pandas as pd
import time
import glob
import torch.optim as optim
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
print(torch.__version__)
import cv2

1.8.0


In [2]:
import torch
t = torch.tensor(np.random.random_sample((3,200,100)))
# functional.crop(t,100,200,200,200)
based_t = torch.zeros((3,500,300))
based_t[:,:200,:100] = t
print(based_t.size(1))
print(t.size(1))
print(based_t[0])

500
200
tensor([[0.9725, 0.6133, 0.1102,  ..., 0.0000, 0.0000, 0.0000],
        [0.2616, 0.2720, 0.1264,  ..., 0.0000, 0.0000, 0.0000],
        [0.1939, 0.4648, 0.2133,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


In [5]:
import torchvision.transforms
from torchvision.transforms import functional
from tqdm import tqdm
from PIL import Image
func_crop = functional.crop

def cut_img_png(img_path_list, save_path, stride=128,img_size = 256):
    toimage = transforms.ToPILImage()
    totensor = transforms.ToTensor()
    os.makedirs(f'{save_path}{img_size}', exist_ok=True)

    for path in tqdm(img_path_list):
        num=0
        img = Image.open(path)
        img = totensor(img)
        for top in range(0, img.size()[2], stride):
            for left in range(0, img.size()[1], stride):
                based_tensor = torch.zeros((3,img_size,img_size))
                arr_cropped = func_crop(img,top,left,img_size,img_size)
                w, h = arr_cropped.size(1), arr_cropped.size(2)
                based_tensor[:, :w, :h] = arr_cropped
                to_pil = toimage(based_tensor)
                title = path[path.rfind('/'):]
                # title = path[:path.rfind('/')]
                to_pil.save(f'{save_path}/{img_size}/{title}_{num}.png')
                num+=1
                
def cut_img_npy(img_path_list, save_path, stride=128,img_size = 256):
    os.makedirs(f'{save_path}/{img_size}', exist_ok=True)
    for path in tqdm(img_path_list):
        num = 0
        img = cv2.imread(path)
        for top in range(0, img.shape[0], stride):
            for left in range(0, img.shape[1], stride):
                piece = np.zeros([img_size, img_size, 3], np.uint8)
                temp = img[top:top+img_size, left:left+img_size, :]
                piece[:temp.shape[0], :temp.shape[1], :] = temp
                title = path[path.rfind('/'):]
                filename= f'{save_path}/{img_size}/{title}_{num}.npy'
                # if not os.path.isfile(filename):
                np.save(filename,piece)
                num+=1

def cut_img_half(img_path_list, save_path):
    os.makedirs(f'{save_path}half', exist_ok=True)
    num = 0
    for path in tqdm(img_path_list):
        img = cv2.imread(path)
        len_half_x = img.shape[1]//2
        ratio_len = img.shape[0]/img.shape[1]
        len_half_y = img.shape[0]//2
        for top in range(0, img.shape[0], len_half_y//2):
            for left in range(0, img.shape[1], len_half_x//2):
                piece = np.zeros([len_half_y, len_half_x, 3], np.uint8)
                temp = img[top:top+len_half_y, left:left+len_half_x, :]
                piece[:temp.shape[0], :temp.shape[1], :] = temp
                np.save(f'{save_path}half/{num}.npy', piece)
                num+=1

def cut_img_6devide(img_path_list, save_path):
    from tqdm import tqdm
    os.makedirs(f'{save_path}6devide', exist_ok=True)
    num = 0
    for path in tqdm(img_path_list):
        img = cv2.imread(path)
        len_x = img.shape[1]//6
        len_y = img.shape[0]//6
        for top in range(0, img.shape[0], len_y//2):
            for left in range(0, img.shape[1], len_x//2):
                piece = np.zeros([len_y, len_x, 3], np.uint8)
                temp = img[top:top+len_y, left:left+len_x, :]
                piece[:temp.shape[0], :temp.shape[1], :] = temp
                np.save(f'{save_path}6devide/{num}.npy', piece)
                num+=1                
                

def cut_img_fullscreen(img_path_list, save_path):
    from tqdm import tqdm
    os.makedirs(f'{save_path}full', exist_ok=True)
    num = 0
    for path in tqdm(img_path_list):
        img = cv2.imread(path)
        len_x = img.shape[1]
        len_y = img.shape[0]
        np.save(f'{save_path}full/{num}_full.npy', img)
        num+=1                

In [9]:
cut_img_npy(list_path, '/media/data1/mhkim/SR/DOTA2.0/train/images')
cut_img_npy(list_path_2, '/media/data1/mhkim/SR/DOTA2.0/train/images_x2_resize')


100%|██████████| 2792/2792 [30:37<00:00,  1.52it/s]  


In [8]:
path_1 = '/home/mhkim/SuperResolution/dataset/DOTAv20/test/images/'
path_2 = '/home/mhkim/SuperResolution/dataset/DOTAv20/test/images_x2_resize/'

list_path = []
list_path_2=[]

import os
cnt=0
for (a,b,c) in os.walk(path_1):
    for _c in c:
        if 'png' in _c:
            list_path.append(os.path.join(a,_c))
            
import os
cnt=0
for (a,b,c) in os.walk(path_2):
    for _c in c:
        if 'png' in _c:
            list_path_2.append(os.path.join(a,_c))            

len(list_path) == len(list_path_2)

# len(list_path)

True

In [4]:
import pandas as pd
train_csv = pd.read_csv('./camera_dataset/train.csv')
test_csv = pd.read_csv('./camera_dataset/test.csv')
train_all_input_files = './camera_dataset/train_input_img/'+train_csv['input_img']
train_all_label_files = './camera_dataset/train_label_img/'+train_csv['label_img']
train_input_files = train_all_input_files[120:].to_numpy()
train_label_files = train_all_label_files[120:].to_numpy()

val_input_files = train_all_input_files[:120].to_numpy()
val_label_files = train_all_label_files[:120].to_numpy()

In [ ]:
name_root = '/home/mhkim/SuperResolution/camera_dataset'
cut_img(train_input_files, f'{name_root}/train_input_img_')

#### NUMPY TO PNG AND SAVE (func : os.makedirs / glob.glob / np.load / Image.fromarray(PIL to NUMPY))

In [33]:
from glob import glob
name_root = '/home/mhkim/SuperResolution/camera_dataset'
name_folder = '/home/mhkim/SuperResolution/camera_dataset/train_label_img_512'
img_path_list = glob(os.path.join(name_folder,'*'))
img_path_list_name = os.listdir(name_folder)

img_path_list_name_only = [name.split('.')[0] for name in img_path_list_name]

cnt = 0
name_saved = os.path.join(name_root,'train_label_img_512_png')
os.makedirs(name_saved,exist_ok=True)
for path in tqdm(img_path_list):
    path = np.load(path)
    new_im = Image.fromarray(path)
    name = img_path_list_name_only[cnt]
    new_im.save(f'{name_saved}/{name}.png')
    cnt+=1


100%|██████████| 34153/34153 [12:02<00:00, 47.24it/s]
